# Import thư viện

In [52]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# LOAD DATA

In [53]:
df = pd.read_csv("data\\raw\\movies_dataset_revenue.csv")
target = "revenue"

# Fill missing, chuẩn hoá dạng datetime

In [54]:
# Date
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["release_year"] = df["release_date"].dt.year

# Thuộc tính kiểm tra phim có ra mắt vào tháng cao điểm hay không
blockbuster_months = [5, 6, 7, 11, 12]
df["is_holiday_release"] = df["release_date"].apply(
    lambda x: 1 if x and (x.month in blockbuster_months) else 0
)

# Fill cột text
text_cols = [
    "cast", "director", "production_companies", "production_countries", "keywords",
    "genres", "collection", "original_language"
    ]
for col in text_cols:
    df[col] = df[col].fillna("")

# Fill cột số liệu bằng median
num_cols = ["budget", "runtime", "vote_count", "popularity", "rating", "revenue"]
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# Split train/test
Để tránh làm mô hình "nhìn" trước được tương lai, tức là dữ liệu tập train được encode bằng dữ liệu của tập test

In [55]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Lấy log của doanh thu để phân phối tốt hơn


In [56]:
train["revenue_log"] = np.log1p(train[target])
test["revenue_log"] = np.log1p(test[target])
target = "revenue_log"

# Target encoding function
Hàm này sẽ giúp thay encode các thuộc tính text bằng trung bình doanh thu tương ứng (ví dụ cột diễn viên sẽ thay bằng trung bình doanh thu của của các phim mà diễn viên đó đã tham gia)

In [57]:
def mean_encoding(train_df, column, target):
    table = defaultdict(list)
    for _, row in train_df.iterrows():
        items = [v.strip() for v in row[column].split(",") if v.strip()]
        for item in items:
            table[item].append(row[target])
    mean_map = {k: np.mean(v) for k, v in table.items()}
    global_mean = train_df[target].mean()

    def encoder(series):
        result = []
        for cell in series:
            items = [x.strip() for x in cell.split(",") if x.strip()]
            if not items:
                result.append(global_mean)
            else:
                result.append(np.mean([mean_map.get(i, global_mean) for i in items]))
        return result
    return encoder


# Apply target encoding


In [58]:
enc_cast = mean_encoding(train, "cast", target)
enc_director = mean_encoding(train, "director", target)
enc_comp = mean_encoding(train, "production_companies", target)
enc_p_coun = mean_encoding(train, "production_countries", target)
enc_orig_lang = mean_encoding(train, "original_language", target)
train["cast_score"] = enc_cast(train["cast"])
test["cast_score"]  = enc_cast(test["cast"])

train["director_score"] = enc_director(train["director"])
test["director_score"]  = enc_director(test["director"])

train["company_score"] = enc_comp(train["production_companies"])
test["company_score"]  = enc_comp(test["production_companies"])

train["country_score"] = enc_p_coun(train["production_countries"])
test["country_score"]  = enc_p_coun(test["production_countries"])

train["orig_lang_score"] = enc_orig_lang(train["original_language"])
test["orig_lang_score"]  = enc_orig_lang(test["original_language"]) 

# One-hot encoding cho production_countries, original_language

# Multi-hot encoding cho genres

In [59]:
# Danh sach thể loại
train["genre_list"] = train["genres"].apply(lambda x: [i.strip() for i in x.split(",") if i])
test["genre_list"] = test["genres"].apply(lambda x: [i.strip() for i in x.split(",") if i])

# Chỉ chia cột thể loại thành N thể loại phố biến nhất còn lại gộp vào "other"
from collections import Counter
N = 10   
all_genres = []
train["genre_list"].apply(lambda x: all_genres.extend(x))
top_genres = [g for g, _ in Counter(all_genres).most_common(N)]

# Hàm refine giúp chia thể loại thành top N và Others
def refine(text):
    refined = []
    for g in text:
        if g in top_genres:
            refined.append(g)
        else:
            refined.append("Others")
    return list(set(refined))

# Genres
train["genres_refined"] = train["genre_list"].apply(refine)
test["genres_refined"] = test["genre_list"].apply(refine)
train["genres_text"] = train["genres_refined"].apply(lambda x: " ".join(x))
test["genres_text"] = test["genres_refined"].apply(lambda x: " ".join(x))

vec = CountVectorizer(binary=True)
Xg_train = vec.fit_transform(train["genres_text"])
Xg_test  = vec.transform(test["genres_text"])

genres_train = pd.DataFrame(
    Xg_train.toarray(),
    columns=[f"genre_{g}" for g in vec.get_feature_names_out()]
)
genres_test = pd.DataFrame(
    Xg_test.toarray(),
    columns=[f"genre_{g}" for g in vec.get_feature_names_out()]
)

train = pd.concat([train.reset_index(drop=True), genres_train], axis=1)
test  = pd.concat([test.reset_index(drop=True), genres_test], axis=1)

# Drop raw text

In [ ]:
# Drop cột keywords vì thiếu nhiều và tác dụng thì khá giống genres
drop_cols = [
    "cast","director","production_companies","genres","genre_list", "revenue",
    "genres_refined","genres_text","keywords","release_date", "id", "title"
]

train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)

# Chuẩn hoá các dữ liệu số

In [ ]:
scale_cols = ["budget","runtime","vote_count","popularity","rating",
              "cast_score","director_score","company_score", "revenue_log"]

scaler = MinMaxScaler()
train[scale_cols] = scaler.fit_transform(train[scale_cols])
test[scale_cols] = scaler.transform(test[scale_cols])

# Lưu dữ liệu để chuẩn bị train

In [62]:
def save_data(df, filename):
    df.to_csv(f"data\\processed\\{filename}", index=False)

save_data(train, "train_processed.csv")
save_data(test, "test_processed.csv")